Let's create a comprehensive Docker guide. Here's the step-by-step plan:

1. Basic Dockerfile structure
2. Docker commands
3. Docker Compose setup
4. Docker networking
5. Multi-stage builds
6. Best practices

### 1. Basic Dockerfile



In [ ]:
# Base image
FROM node:18-alpine

# Working directory
WORKDIR /app

# Copy package files
COPY package*.json ./

# Install dependencies
RUN npm install

# Copy source code
COPY . .

# Expose port
EXPOSE 3000

# Start command
CMD ["npm", "start"]



### 2. Essential Docker Commands



In [ ]:
# Build image
docker build -t myapp:latest .

# Run container
docker run -d -p 3000:3000 myapp:latest

# List containers
docker ps

# Stop container
docker stop <container_id>

# Remove container
docker rm <container_id>

# List images
docker images

# Remove image
docker rmi <image_id>



### 3. Docker Compose Setup



In [ ]:
# docker-compose.yml
version: '3.8'

services:
  web:
    build: .
    ports:
      - "3000:3000"
    environment:
      - NODE_ENV=production
    depends_on:
      - db
  
  db:
    image: postgres:13
    environment:
      - POSTGRES_PASSWORD=secret
    volumes:
      - postgres_data:/var/lib/postgresql/data

volumes:
  postgres_data:



### 4. Docker Network Commands



In [ ]:
# Create network
docker network create mynetwork

# List networks
docker network ls

# Connect container to network
docker network connect mynetwork container_name

# Inspect network
docker network inspect mynetwork



### 5. Multi-stage Build Example



In [ ]:
# Build stage
FROM node:18-alpine AS builder
WORKDIR /app
COPY package*.json ./
RUN npm install
COPY . .
RUN npm run build

# Production stage
FROM nginx:alpine
COPY --from=builder /app/build /usr/share/nginx/html
EXPOSE 80
CMD ["nginx", "-g", "daemon off;"]



### 6. Best Practices



In [ ]:
# Use specific versions
FROM node:18.15.0-alpine3.17

# Layer caching
COPY package*.json ./
RUN npm ci

# Security
USER node

# Health check
HEALTHCHECK --interval=30s \
  CMD curl -f http://localhost:3000/health || exit 1

# Labels
LABEL maintainer="dev@example.com" \
      version="1.0" \
      description="Node.js application"

# Comprehensive Docker Guide - Step by Step Plan

1. Base Docker Setup
   - Dockerfile fundamentals
   - Basic container management

2. Container Orchestration
   - Docker Compose multi-service setup
   - Network configuration

3. Development Workflow
   - Multi-stage builds
   - Development vs Production environments

4. Best Practices & Security
   - Image optimization
   - Security considerations

Let's implement each step:

### 1. Base Docker Setup



In [ ]:


FROM node:18-alpine

WORKDIR /app

COPY package*.json ./
RUN npm ci --only=production

COPY . .

EXPOSE 3000
USER node

CMD ["npm", "start"]



### 2. Container Orchestration



In [ ]:


version: '3.8'

services:
  api:
    build: 
      context: .
      target: development
    volumes:
      - .:/app
      - /app/node_modules
    ports:
      - "3000:3000"
    environment:
      - NODE_ENV=development
    networks:
      - app-network

  db:
    image: postgres:14-alpine
    environment:
      POSTGRES_PASSWORD: ${DB_PASSWORD:-secret}
    volumes:
      - db-data:/var/lib/postgresql/data
    networks:
      - app-network

networks:
  app-network:
    driver: bridge

volumes:
  db-data:



### 3. Development Workflow



In [ ]:


FROM node:18-alpine AS dependencies
WORKDIR /app
COPY package*.json ./
RUN npm ci

FROM node:18-alpine AS builder
WORKDIR /app
COPY --from=dependencies /app/node_modules ./node_modules
COPY . .
RUN npm run build

FROM node:18-alpine AS production
WORKDIR /app
COPY --from=dependencies /app/package*.json ./
RUN npm ci --only=production
COPY --from=builder /app/dist ./dist
USER node
CMD ["node", "dist/main.js"]



### 4. Command Reference



In [ ]:
# Build image
docker build -t myapp:dev .

# Run containers
docker-compose up -d

# View logs
docker-compose logs -f

# Execute tests
docker-compose exec api npm test

# Cleanup
docker-compose down -v



### 5. Security Configuration



In [ ]:


node_modules
npm-debug.log
Dockerfile*
docker-compose*
.git
.env
*.md

# Docker Implementation Plan

1. Project Structure
   - Base Dockerfile
   - Development/Production configs
   - Docker Compose services
   - Docker ignore and security

2. Core Services Setup
   - Web application
   - Database
   - Cache layer
   - Message queue

3. Development Environment  
   - Hot reloading
   - Debug configuration
   - Volume mounts

4. Production Environment
   - Multi-stage builds
   - Security hardening
   - Health checks
   - Logging

5. CI/CD Pipeline
   - Build stages
   - Testing
   - Deployment

### Base Implementation



In [ ]:


FROM node:18-alpine AS base

# Security: Run as non-root
RUN addgroup -S appgroup && adduser -S appuser -G appgroup
USER appuser

WORKDIR /app

# Layer optimization
COPY package*.json ./
RUN npm ci --only=production

COPY . .

# Health check
HEALTHCHECK --interval=30s \
  CMD wget --no-verbose --tries=1 --spider http://localhost:3000/health || exit 1

EXPOSE 3000
CMD ["npm", "start"]

In [ ]:


version: '3.8'

services:
  web:
    build: 
      context: .
      target: development
    volumes:
      - .:/app
      - /app/node_modules
    ports:
      - "3000:3000"
    depends_on:
      - db
      - redis
      - rabbitmq

  db:
    image: postgres:14-alpine
    environment:
      POSTGRES_PASSWORD: ${DB_PASSWORD}
    volumes:
      - pgdata:/var/lib/postgresql/data

  redis:
    image: redis:alpine
    ports:
      - "6379:6379"

  rabbitmq:
    image: rabbitmq:management-alpine
    ports:
      - "5672:5672"
      - "15672:15672"

volumes:
  pgdata:

networks:
  app-network:
    driver: bridge

In [ ]:


node_modules
npm-debug.log
.git
.env
*.md
.DS_Store
tests
docs







Notes for next iterations:
1. Add monitoring setup (Prometheus/Grafana)
2. Implement logging (ELK stack)
3. Add backup strategies
4. Document scaling approaches
5. Create deployment templates